# Analise de notas dos alunos da UFRN

## 1. Introdução

O objetivo desse *notebook* é realizar uma série de correlações sobre as turmas ofertadas pelo Instituto Metrópole Digital (IMD), departamento da Universidade Federal do Rio Grande do Norte (UFRN), para avaliar se o horário dessas turmas tem algum efeito no sucesso ou insucesso do aluno, se o professor também tem algum peso nessas chances e etc.

Para isso foram escolhidas algumas das disciplinas com maior porcentagem de insucesso são elas: Fundamentos Matemáticos da Computação I (FMCI), Fundamentos Matemáticos da Computação II (FMCII), Cálculo I além da disciplina deste projeto Probabilidade. Todos os dados utilizados para análise foram retirados do banco de dados público da UFRN no *website* http://dados.ufrn.br.  

## 2 Recursos utilizados

Na análise dos dados serão utilizados as seguintes bibliotecas *python* o *Pandas* para a criação e manipulção dos *dataframes*, *Numpy* para a análise matemática e o *Bokeh* para a visualização dos dados.

## 3 Preparação do Ambiente e Criação dos Dataframes

Antes de começarmos é necessário preparar o *notebook*, instalado as bibliotecas já citadas rodando o seguinte comando, e realizar a criação dos dados e subsequentemente manipulá-los para facilitar a sua análise. Todo o processo de manipulção está descrito em forma de commentários nos códigos.

In [1]:
!pip install pandas
!pip install numpy
!pip install bokeh

notebook 5.4.0 requires terminado>=0.8.1, which is not installed.


notebook 5.4.0 requires terminado>=0.8.1, which is not installed.


notebook 5.4.0 requires terminado>=0.8.1, which is not installed.


In [1]:
# Inclusão das bibliotecas utilizadas
import pandas as pd
import numpy as np
import bokeh
from urllib.request import urlopen

In [38]:
# Setup das base de dados a serem utilizadas.

url_componentes_curriculares = 'http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv'

url_cursos_ufrn = 'http://dados.ufrn.br/dataset/08b0dc59-faa9-4281-bd1e-2a39f532489e/resource/949be3d1-e85b-4d0f-9f60-1d9a7484bb06/download/cursos-ufrn'

url_matriculas_2015_1 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/9e7ba1c2-f92d-4b9c-9e91-3b026ecdf913/download/matriculas-de-2015.1.csv'
url_matriculas_2015_2 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/baa6c8b4-2072-417f-b238-c028ccc8c14b/download/matriculas-de-2015.2.csv'
url_matriculas_2016_1 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/4778d3ce-8898-46a8-a623-ee6a480a2980/download/matriculas-de-2016.1.csv'
url_matriculas_2016_2 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/f6179838-b619-4d7d-af9c-18c438b80dd4/download/matriculas-de-2016.2.csv'
url_matriculas_2017_1 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/79071c21-e32c-438f-b930-d1b6ccc02ec2/download/matricula-componente-20171'
url_matriculas_2017_2 = 'http://dados.ufrn.br/dataset/c8650d55-3c5a-4787-a126-d28a4ef902a6/resource/55dfe713-ff7c-4fa8-8d1d-d4294a025bff/download/matricula-componente-20172'

url_turmas_2015_1 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/4d5aee5a-00b0-4ed6-a4be-59fa77a56797/download/turmas-2015.1.csv'
url_turmas_2015_2 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/7c59621c-4a8b-49d4-b319-83cfea9bdf28/download/turmas-2015.2.csv'
url_turmas_2016_1 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/322d9977-ba15-47f1-8216-75a1ca78e197/download/turmas-2016.1.csv'
url_turmas_2016_2 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e8e3228-7f22-40a2-9efd-561c44844567/download/turmas-2016.2.csv'
url_turmas_2017_1 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/5e77d066-d506-45eb-a21e-76aa79616fef/download/turmas-2017.1.csv'
url_turmas_2017_2 = 'http://dados.ufrn.br/dataset/1938623d-fb07-41a4-a55a-1691f7c3b8b5/resource/01fe7343-fdf0-4a67-b915-2386b7c2fecb/download/turmas-2017.2.csv'

docentes_url = 'http://dados.ufrn.br/dataset/8bf1a468-48ff-4f4d-95ee-b17b7a3a5592/resource/ff0a457e-76fa-4aca-ad99-48aebd7db070/download/docentes.csv'

In [39]:
# Leitura dos csvs docentes, componentes, matriculas e cursos.

docentes = pd.read_csv(docentes_url, sep=';')

cursos_ufrn = pd.read_csv(url_cursos_ufrn, sep=';', error_bad_lines=False)

componente = pd.read_csv(url_componentes_curriculares, sep=';'
                         , usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

matriculas_2015_1 = pd.read_csv(url_matriculas_2015_1, sep=';')
matriculas_2015_2 = pd.read_csv(url_matriculas_2015_2, sep=';')
matriculas_2016_1 = pd.read_csv(url_matriculas_2016_1, sep=';')
matriculas_2016_2 = pd.read_csv(url_matriculas_2016_2, sep=';')
matriculas_2017_1 = pd.read_csv(url_matriculas_2017_1, sep=';')
matriculas_2017_2 = pd.read_csv(url_matriculas_2017_2, sep=';')

In [49]:
# Leitura dos csvs das turmas.

turmas_2015_1 = pd.read_csv(url_turmas_2015_1, sep=';')
turmas_2015_2 = pd.read_csv(url_turmas_2015_2, sep=';')
turmas_2016_1 = pd.read_csv(url_turmas_2016_1, sep=';') 
turmas_2016_2 = pd.read_csv(url_turmas_2016_2, sep=';')
turmas_2017_1 = pd.read_csv(url_turmas_2017_1, sep=';')
turmas_2017_2 = pd.read_csv(url_turmas_2017_2, sep=';')

turmas_2015_1.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2015_2.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2016_1.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2016_2.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2017_1.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas_2017_2.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

turmas_2015_1.id_componente = turmas_2015_1.id_componente.astype(str)
turmas_2015_2.id_componente = turmas_2015_2.id_componente.astype(str)
turmas_2016_1.id_componente = turmas_2016_1.id_componente.astype(str)
turmas_2016_2.id_componente = turmas_2016_2.id_componente.astype(str)
turmas_2017_1.id_componente = turmas_2017_1.id_componente.astype(str)
turmas_2017_2.id_componente = turmas_2017_2.id_componente.astype(str)

# Adicionando 

turmas_2015_1['ano_semestre'] = '2015_1'
turmas_2015_2['ano_semestre'] = '2015_2'
turmas_2016_1['ano_semestre'] = '2016_1'
turmas_2016_2['ano_semestre'] = '2016_2'
turmas_2017_1['ano_semestre'] = '2017_1'
turmas_2017_2['ano_semestre'] = '2017_2'

In [43]:
# Retirando a coluna Unnamed: 10 de alguns csvs

matriculas_2015_1.drop('Unnamed: 10', 1, inplace=True) 
matriculas_2015_2.drop('Unnamed: 10', 1, inplace=True) 
matriculas_2016_1.drop('Unnamed: 10', 1, inplace=True) 
matriculas_2016_2.drop('Unnamed: 10', 1, inplace=True) 

# Criando coluna ano_semestre

matriculas_2015_1['ano_semestre'] = '2015_1'
matriculas_2015_2['ano_semestre'] = '2015_2'
matriculas_2016_1['ano_semestre'] = '2016_1'
matriculas_2016_2['ano_semestre'] = '2016_2'
matriculas_2017_1['ano_semestre'] = '2017_1'
matriculas_2017_2['ano_semestre'] = '2017_2'

In [50]:
# Combinando dataframes das matrículas.

frames = [matriculas_2015_1, matriculas_2015_2, matriculas_2016_1, matriculas_2016_2, matriculas_2017_1, matriculas_2017_2]
matriculas = pd.concat(frames)

# Combinando os dataframes das turmas.

frames  = [turmas_2015_1, turmas_2015_2, turmas_2016_1, turmas_2016_2, turmas_2017_1, turmas_2017_2]
turmas = pd.concat(frames)

In [52]:
# Limpando dados inválidos/desconhecidos da descrição.

matriculas = matriculas[matriculas.descricao != 'DESISTENCIA']
matriculas = matriculas[matriculas.descricao != 'CANCELADO']
matriculas = matriculas[matriculas.descricao != 'INDEFERIDO']
matriculas = matriculas[matriculas.descricao != 'EXCLUIDA']
matriculas = matriculas[matriculas.descricao != 'DISPENSADO']
matriculas = matriculas[matriculas.descricao != 'CUMPRIU']
matriculas = matriculas[matriculas.descricao != 'EM ESPERA']
matriculas = matriculas[matriculas.descricao != 'AGUARDANDO DEFERIMENTO']
matriculas = matriculas[matriculas.descricao != 'TRANSFERIDO']
matriculas = matriculas[matriculas.descricao != 'INCORPORADO']

In [53]:
# Retirando valores NaN do dataframe.

componente.dropna(inplace=True)
componente = componente[~componente.id_componente.duplicated()]


In [54]:
# Gerando dataframe com as turmas e componentes.

colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario", "ano_semestre"]
merge_turmas_componentes = pd.merge(turmas, componente, on='id_componente')[colunas]

merge_turmas_componentes.dropna(inplace=True)


In [55]:
# Filtrando para os componentes do IMD

componentes_imd = merge_turmas_componentes[merge_turmas_componentes.codigo.str.contains('IMD')]

componentes_imd = componentes_imd[componentes_imd.situacao_turma == 'CONSOLIDADA']

# Alterando o nome da coluna id_docente_interno para id_servidor (mesma do dataframe docentes).

componentes_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)
componentes_imd.capacidade_aluno = componentes_imd.capacidade_aluno.astype(np.int64)
componentes_imd.id_servidor = componentes_imd.id_servidor.astype(np.int64)


In [56]:
# Merge dos dataframes componentes_imd e docentes, para amarrar os nomes dos docentes as turmas.

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario", "ano_semestre"]

turmas_imd = pd.merge(componentes_imd, docentes, on="id_servidor")[colunas]
turmas_imd.id_turma = turmas_imd.id_turma.astype(np.int64)
turmas_imd.drop('ano_semestre', axis=1, inplace=True)



In [59]:
# Merge dos dataframes matriculas e turmas_imd.

matriculas.id_turma = matriculas.id_turma.astype(np.int64)
colunas = ["id_turma", "discente", "codigo", "nome_y", "nome_x", "descricao_horario", "unidade", "nota", "reposicao"
           , "media_final", "numero_total_faltas", "descricao", "ano_semestre"]

merge_matriculas_turmas_imd = pd.merge(matriculas, turmas_imd, on="id_turma")[colunas]

In [60]:
merge_matriculas_turmas_imd.head()

,id_turma,discente,codigo,codigo,nome_y,nome_x,descricao_horario,unidade,nota,reposicao,media_final,numero_total_faltas,descricao,ano_semestre
0,1239422,bcd9e4d861acc00fac216bbe953c63b8,IMD0040,IMD0040,CARLOS EDUARDO DA SILVA,LINGUAGEM DE PROGRAMAÇÃO II,24N12 (02/02/2015 - 20/06/2015),2.0,7.6,f,7.9,2.0,APROVADO,2015_1
1,1239422,bcd9e4d861acc00fac216bbe953c63b8,IMD0040,IMD0040,CARLOS EDUARDO DA SILVA,LINGUAGEM DE PROGRAMAÇÃO II,24N12 (02/02/2015 - 20/06/2015),1.0,8.4,f,7.9,2.0,APROVADO,2015_1
2,1239422,bcd9e4d861acc00fac216bbe953c63b8,IMD0040,IMD0040,CARLOS EDUARDO DA SILVA,LINGUAGEM DE PROGRAMAÇÃO II,24N12 (02/02/2015 - 20/06/2015),3.0,7.6,f,7.9,2.0,APROVADO,2015_1
3,1239422,1fdfae54f2e3d6e8a00d8ceab48c74f6,IMD0040,IMD0040,CARLOS EDUARDO DA SILVA,LINGUAGEM DE PROGRAMAÇÃO II,24N12 (02/02/2015 - 20/06/2015),1.0,7.3,f,6.0,18.0,APROVADO POR NOTA,2015_1
4,1239422,1fdfae54f2e3d6e8a00d8ceab48c74f6,IMD0040,IMD0040,CARLOS EDUARDO DA SILVA,LINGUAGEM DE PROGRAMAÇÃO II,24N12 (02/02/2015 - 20/06/2015),2.0,5.4,f,6.0,18.0,APROVADO POR NOTA,2015_1


In [26]:
merge_matriculas_turmas_imd.to_csv('matriculas_finais.csv', sep=';')

In [74]:
# Criação dos dataframes contendo as notas das disciplinas especificadas.

fmcI = merge_matriculas_turmas_imd.loc[merge_matriculas_turmas_imd['nome_x'] == 'FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO I']
fmcII = merge_matriculas_turmas_imd.loc[merge_matriculas_turmas_imd['nome_x'] == 'FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO II']
calc = merge_matriculas_turmas_imd.loc[merge_matriculas_turmas_imd['nome_x'] == 'CÁLCULO DIFERENCIAL E INTEGRAL I']
prob = merge_matriculas_turmas_imd.loc[merge_matriculas_turmas_imd['nome_x'] == 'PROBABILIDADE']

## 4 Visualização e Análise dos Dados

### 4.1 